In [1]:
import torch
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
def evaluate_explainability_robustness(model, instance, explain_func, perturbation_scale=0.01, num_perturbations=50):
    """
    Evaluate the robustness of an explainability vector for a given instance and model.

    Parameters:
    - model: torch.nn.Module
        The PyTorch model to be evaluated.
    - instance: torch.Tensor
        The input instance for which the explainability vector is computed (1D tensor).
    - explain_func: function
        A function that takes `model` and `instance` as inputs and returns an explainability vector.
    - perturbation_scale: float
        The standard deviation of Gaussian noise added to the input for perturbations.
    - num_perturbations: int
        The number of perturbed instances to generate.

    Returns:
    - robustness_score: float
        A score indicating the robustness of the explainability vector (lower is better).
    """
    model.model.eval()  # Ensure the model is in evaluation mode
    if not isinstance(instance, torch.Tensor):
        instance = torch.tensor(instance, dtype=torch.float32)
    # Compute the original explainability vector
    original_explain_vector = torch.tensor(explain_func(instance))

    # Initialize a list to store differences in explainability vectors
    explain_diff_magnitudes = []

    for _ in tqdm(range(num_perturbations)):
        # Generate a perturbed instance by adding Gaussian noise
        perturbation = torch.randn_like(instance) * perturbation_scale
        perturbed_instance = instance + perturbation

        # Compute the explainability vector for the perturbed instance
        perturbed_explain_vector = torch.tensor(explain_func(perturbed_instance))

        # Compute the difference between the original and perturbed explainability vectors
        diff = original_explain_vector - perturbed_explain_vector

        # Measure the magnitude of the difference
        diff_magnitude = torch.norm(diff).item()
        explain_diff_magnitudes.append(diff_magnitude)

    # Compute the average magnitude of differences as the robustness score
    robustness_score = np.mean(explain_diff_magnitudes)

    return robustness_score


In [3]:
experiment_path = "../results/all_db_all_training/DTEC_DSIL_deterministic_exponential_s0_T400_bins7/A_synthetic_f4_s5000_c2_r0.05_0.05_seed_0"

model_path = f"{experiment_path}/model.pth"

model = torch.load(model_path, weights_only=True)

dataset_path = f"{experiment_path}/dataset.pth"


In [4]:
experiment_config_path = Path(f"{experiment_path}/experiment_config.yaml")
with open(experiment_config_path, "r") as file:
    experiment_config = yaml.safe_load(file)

dataset_path = f"../{experiment_config['dataset']['dataset_path']}"
dataset_path

'../dataset/synthetic_generation/A_synthetic_f4_s5000_c2_r0.05_0.05_seed_0/data.npy'

In [5]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
sys.path.append(str(project_root))
sys.path.append("../")


from src.utils import get_dataset, select_model

2025-01-20 13:35:51.561345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 13:35:51.575436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 13:35:51.579676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 13:35:51.590754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 13:35:52.392494: W tensorflow/comp

In [6]:
from hydra import initialize, compose

# Initialize Hydra and load the configuration
with initialize(config_path=experiment_path):
    cfg = compose(config_name=experiment_config_path.name)
cfg.dataset.dataset_path = "../" + cfg.dataset.dataset_path

/tmp/ipykernel_2103255/2641106970.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=experiment_path):


In [ ]:
dataset = get_dataset(cfg)

{'Samples': 5000, 'Features': 4, 'Anomalies': 500, 'Anomalies Ratio(%)': 10.0}


In [8]:
X = dataset['X_test']
y = dataset['y_test']
explanation = dataset['explanation_test']

# keep only data with label != 0
X = X[y != 0]
explanation = explanation[y != 0]
y = y[y != 0]

# Select 200 random instances from the dataset
num_instances = 200
random_indices = np.random.choice(len(X), num_instances, replace=False)
instances = X[random_indices]
explanations = explanation[random_indices]
labels = y[random_indices]

In [9]:
# Evaluate the robustness of the explanation for the first instance
model = select_model(cfg.model, device="cuda:0" if torch.cuda.is_available() else "cpu")
model.load_model(model_path, X)

/home/elouan/code/code_repo/src/models/dte.py:416: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(path))


In [10]:
instances[0].shape

(4,)

In [11]:
from src.shap_explainer import ShapExplainer

explainer = ShapExplainer(model, X)


In [12]:
shap_robustness = evaluate_explainability_robustness(model, instances[0].reshape(1,-1), explainer.explain_instance, perturbation_scale=0.01, num_perturbations=50)

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
ours_robustness = evaluate_explainability_robustness(model, instances[0].reshape(1,-1), model.instance_explanation, perturbation_scale=0.01, num_perturbations=50)

/home/elouan/code/code_repo/src/models/dte.py:327: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(x[:, i]),


  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
gradient_robustness = evaluate_explainability_robustness(model, instances[0].reshape(1,-1), model.gradient_explanation, perturbation_scale=0.01, num_perturbations=50)

TypeError: expected np.ndarray (got Tensor)

In [14]:
ours_robustness - shap_robustness

-0.00021756765591168727

In [ ]:
ours_robustness, shap_robustness, gradient_robustness